# E2E 테스트 – FAQ-only RAG PoC  

1. '.env' 로부터 OpenAI 키 로딩  
2. FAQ JSONL → FAISS 인덱스 생성  
3. LangGraph FAQ 파이프라인 로드  
4. 샘플 질의에 대한 응답 생성  
5. RAGAS 평가로 정확도·신뢰도 확인

In [1]:
# 1. env & 라이브러리
from pathlib import Path
from dotenv import load_dotenv
import os, json, pprint, sys, pandas as pd

ROOT = Path.cwd().parent  # 노트북은 tests/ 하위에 있다고 가정
sys.path.append(str(ROOT))

load_dotenv(ROOT / ".env")
print(f"ROOT - {ROOT}")
print("환경 변수 로드 완료")


ROOT - c:\Users\insung\Finance_Agent
환경 변수 로드 완료


In [3]:
# 2. 인덱스 준비
from scripts.build_faq_index import build_index

build_index()

2025-07-29 18:58:28,136 | INFO | loading data from C:\Users\insung\Finance_Agent\data\processed\faq_woori_structured.jsonl
2025-07-29 18:58:28,141 | INFO | embedding 99 documents ...
2025-07-29 18:58:31,700 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 18:58:32,533 | INFO | Loading faiss with AVX2 support.
2025-07-29 18:58:32,579 | INFO | Successfully loaded faiss with AVX2 support.
2025-07-29 18:58:32,590 | INFO | Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
2025-07-29 18:58:32,612 | INFO | FAISS index saved → C:\Users\insung\Finance_Agent\index\faq_faiss


In [ ]:
# 3. 파이프라인 로딩
from services.rag.faq_rag.faq_chain import graph

sample_q = "인터넷뱅킹에서 보안매체 없이 이체하려면 어떻게 하나요?"
result = graph.invoke({"question": sample_q})
pprint.pp(result)


2025-07-29 19:01:07,691 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19:01:11,116 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'question': '인터넷뱅킹에서 보안매체 없이 이체하려면 어떻게 하나요?',
 'context': 'Q: 인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야되나요?\n'
            'A: 간편이체 서비스를 가입 후 이용하려는 PC를 지정단말로 등록하면, 직접 설정한 간편이체 한도까지 '
            '보안매체(보안카드/OTP)와 인증서 입력 없이 이체할 수 있습니다.\n'
            '단, 간편이체 한도는 1일 최대 500만원 이내로 설정 가능하며, 전자금융 이체한도를 초과할 수 없습니다.\n'
            '\n'
            '간편이체 서비스는 즉시이체에만 적용되며, 예약이체 또는 지정한 간편이체한도 초과하여 이체, 미지정단말에서 이체 등의 '
            '거래 시에는 적용되지 않습니다. 또한 간편이체 서비스에 의한 거래임에도 안전한 거래로 인식되지 않은 경우 보안매체 '
            '입력을 요구 할 수 있습니다.\n'
            '\n'
            '▶ 간편이체 서비스 경로 안내 : 개인인터넷뱅킹 → 뱅킹관리 → 간편이체 서비스\n'
            '\n'
            '※ 우리WON뱅킹에서는 간편이체서비스가 제공되지 않습니다. 단, 우리WON인증서로 로그인 후 1천만원 이내 이체 '
            '시에는 보안매체 없이 우리WON인증서로 이체 가능하며, 소액이체(300만원 이하)인 경우에는 인증서 입력도 생략 '
            '가능합니다.\n'
            '\n'
            'Q: 인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?\n'
            'A: 인터넷뱅킹에서는 인터넷/스마트뱅킹 이체한도 조회 및 축소는 가능하나 증액은 불가합니다. 이체한도 증액은 '
            '우리WON뱅킹에서 비대면 실명확인 절차로 진행 할 수 있습니다.\n'
        

## 배치 테스트 & RAGAS 평가  
- 10개 샘플 질의를 선택해 파이프라인을 실행  
- answer relevancy / faithfulness / LLMContextPrecisionWithoutReference으로 지표 계산
- 레퍼런스가 없기 때문에 LLMContextPrecisionWithoutReference으로 답변과 컨텐스트 정보의 중복 정도로 precision 평가

In [5]:
# 4. 배치 & RAGAS
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import answer_relevancy, faithfulness, LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 1) 평가용 LLM/임베딩 (Ragas 래퍼로 감싸기)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evaluator_emb = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

# 2) 그래프 호출 → Ragas 샘플로 변환
SAMPLE_QUESTIONS = [
    "인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?",
    "이용자비밀번호 3회 오류 상태인데, 인터넷뱅킹에서 오류 해제할 수 있나요?",
    "해외에서 인터넷/스마트뱅킹을 이용하려면 출국 전에 확인해야 할 점이 있나요?",
    "지정된 기기에서만 인터넷/스마트뱅킹을 이용할 수 있나요?",
    "인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야 하나요?",
    "(개인고객) 인터넷/스마트뱅킹 신규 회원 가입 방법을 알려주세요.",
    "이용자비밀번호(로그인 비밀번호) 2회 오류 상태인데, 인터넷뱅킹에서 재등록할 수 있나요?",
    "해외 거주 중인데 인터넷/스마트뱅킹 비밀번호 오류 해제·서비스 변경은 어떻게 하나요?",
    "공동명의로 통장을 만들 수 있나요?",
    "미성년자 예금 해지 시 필요한 서류가 궁금합니다.",
    "미성년자 예금 가입 시 필요한 서류가 궁금합니다.",
    "미성년자 본인 명의로 예금 가입할 때 부모님 동행이 필요한가요?",
    "신용대출 만기가 다가오는데 연장을 어떻게 신청하나요?",
    "우량협약기업신용대출(PPL) 신청 시 “임직원 정보 갱신 후 대출 진행 가능” 오류가 뜨면 어떻게 해야 하나요?",
    "인터넷뱅킹으로 주택청약종합저축을 담보로 대출할 수 있나요?",
    "예금담보대출 이용 중인데 예금을 해지해 대출금을 상환할 수 있나요?",
    "펀드 계좌에 “이용료”가 입금됐는데 무엇인가요?",
    "적립식 펀드 자동이체 중인데 인출이 안 됐어요. 원인은 무엇인가요?",
    "펀드를 신규 가입했는데 계좌 조회에 보이지 않는 이유가 뭔가요?",
    "펀드 신규·추가 매수가 가능한 시간이 어떻게 되나요?"
]

# 인터넷/스마트뱅킹 FAQ 8문
# 예금 FAQ 4문
# 대출 FAQ 4문
# 펀드 FAQ 4문

samples = []
for q in SAMPLE_QUESTIONS:
    res = graph.invoke({"question": q})
    ans = res.get("answer", "")
    ctxs = res.get("contexts", [])
    if isinstance(ctxs, str):
        ctxs = [c for c in ctxs.split("\n\n") if c.strip()]

    samples.append(
        SingleTurnSample(
            user_input=q,
            response=ans,
            retrieved_contexts=ctxs,
        )
    )

dataset = EvaluationDataset(samples=samples)

# 3) 메트릭 설정 (레퍼런스 없이 가능한 조합)
metrics = [
    answer_relevancy,                                       # Q - A 관련성
    faithfulness,                                           # A가 컨텍스트에 근거?
    LLMContextPrecisionWithoutReference(llm=evaluator_llm)  # 컨텍스트가 Q와 관련?
    ]

# 4) 평가 실행
report = evaluate(
    dataset,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_emb,
    show_progress=True,
    batch_size=8,
)

df_scores  = report.to_pandas()

df_overall = df_scores[[
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]].mean()

print("=== Overall Metrics ===")
print(df_overall)

print("\n=== Sample-level Metrics (sorted by Faithfulness) ===")
print(df_scores.sort_values("faithfulness")[[
    "user_input",
    "retrieved_contexts",
    'response',
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]])

2025-07-29 19:23:23,533 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19:23:26,588 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:23:27,231 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19:23:31,136 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:23:31,991 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19:23:35,951 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:23:36,338 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19:23:40,014 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:23:41,032 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-29 19

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Batch 1/8:   0%|          | 0/8 [00:00<?, ?it/s]

2025-07-29 19:24:59,797 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:24:59,875 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:24:59,947 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,095 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,108 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,156 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,197 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,200 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-29 19:25:00,550 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1

=== Overall Metrics ===
answer_relevancy                           0.355515
faithfulness                               0.922619
llm_context_precision_without_reference    0.975000
dtype: float64

=== Sample-level Metrics (sorted by Faithfulness) ===
                                           user_input  \
15              예금담보대출 이용 중인데 예금을 해지해 대출금을 상환할 수 있나요?   
14                   인터넷뱅킹으로 주택청약종합저축을 담보로 대출할 수 있나요?   
4         인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야 하나요?   
12                      신용대출 만기가 다가오는데 연장을 어떻게 신청하나요?   
3                     지정된 기기에서만 인터넷/스마트뱅킹을 이용할 수 있나요?   
0                   인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?   
2          해외에서 인터넷/스마트뱅킹을 이용하려면 출국 전에 확인해야 할 점이 있나요?   
1           이용자비밀번호 3회 오류 상태인데, 인터넷뱅킹에서 오류 해제할 수 있나요?   
7     해외 거주 중인데 인터넷/스마트뱅킹 비밀번호 오류 해제·서비스 변경은 어떻게 하나요?   
6   이용자비밀번호(로그인 비밀번호) 2회 오류 상태인데, 인터넷뱅킹에서 재등록할 수 있나요?   
5                (개인고객) 인터넷/스마트뱅킹 신규 회원 가입 방법을 알려주세요.   
8                                 공동명의로 통장을 만들 수 있나요?   
11       

In [6]:
for i in range(10):
    print('\nuser_input:',df_scores['user_input'][i])
    print('retrieved_contexts:',df_scores['retrieved_contexts'][i])
    print('response:',df_scores['response'][i])
    print('='*120)


user_input: 인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?
retrieved_contexts: ['Q: 인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?\nA: 인터넷뱅킹에서는 인터넷/스마트뱅킹 이체한도 조회 및 축소는 가능하나 증액은 불가합니다. 이체한도 증액은 우리WON뱅킹에서 비대면 실명확인 절차로 진행 할 수 있습니다.\n\n단, 보안카드 이용자의 경우 증액하려는 이체한도에 따라 OTP발생기로 보안매체 변경이 필요할 수 있습니다.\n\n※ 보안등급별 보안매체 및 이체한도 범위\n\n- 보안 1등급 : OTP발생기, 1회 1억원 이하 / 1일 5억원 이하로 설정 가능\n- 보안 2등급 : 보안카드, 1회 5백만원 이하 / 1일 1천만원 이하로 설정 가능\n- 인터넷뱅킹 이체한도는 스마트뱅킹과 통합 한도입니다.', 'Q: 인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야되나요?\nA: 간편이체 서비스를 가입 후 이용하려는 PC를 지정단말로 등록하면, 직접 설정한 간편이체 한도까지 보안매체(보안카드/OTP)와 인증서 입력 없이 이체할 수 있습니다.\n단, 간편이체 한도는 1일 최대 500만원 이내로 설정 가능하며, 전자금융 이체한도를 초과할 수 없습니다.\n\n간편이체 서비스는 즉시이체에만 적용되며, 예약이체 또는 지정한 간편이체한도 초과하여 이체, 미지정단말에서 이체 등의 거래 시에는 적용되지 않습니다. 또한 간편이체 서비스에 의한 거래임에도 안전한 거래로 인식되지 않은 경우 보안매체 입력을 요구 할 수 있습니다.\n\n▶ 간편이체 서비스 경로 안내 : 개인인터넷뱅킹 → 뱅킹관리 → 간편이체 서비스\n\n※ 우리WON뱅킹에서는 간편이체서비스가 제공되지 않습니다. 단, 우리WON인증서로 로그인 후 1천만원 이내 이체 시에는 보안매체 없이 우리WON인증서로 이체 가능하며, 소액이체(300만원 이하)인 경우에는 인증서 입력도 생략 가능합니다.', 'Q: 인터넷뱅킹을 오랫동안 이용하지 않아서 자금이체가 제한되

- answer_relevancy ≥ 0.8 이면 FAQ 일치율 양호   낮으면 프롬프트/쿼리 확장/리트리버 k값 조정.

- faithfulness < 0.9 라면 Prompt 수정·retriever k값 조정 필요, ‘컨텍스트 내 근거 인용’ 지시 강화, 답변 길이 제한, 시스템 프롬프트 보강.

- context_precision 은 1.0에 가까울수록 불필요한 문맥이 적음. 임베딩 모델/토크나이저, 인덱스 파라미터(HNSW, k), 쿼리 재작성 확인.

 OOD(미지원 질문)용 거절 정확도 같은 보조 지표도 추가하면 좋을 듯.